In [1]:
!pip install ultralytics --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.4 MB/s eta 0:00:00


In [2]:
!ls -l ../input/road-damage-dataset-potholes-cracks-and-manholes/data

total 0
drwxr-xr-x 2 nobody nogroup 0 Dec 16 18:48 images
drwxr-xr-x 2 nobody nogroup 0 Dec 16 18:49 labels


In [3]:
import os
import shutil
import yaml
import gc
from ultralytics import YOLO, settings
import torch
import numpy as np

# Riproducibilità
torch.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Impostazioni generali
PROJECT = "pothole-detector-NatureSR"
RANDOM_SEED = 0
IOU_THRESHOLD = 0.7
CONF_THRESHOLD = 0.25
INPUT_SIZE = 640
PATIENCE = 10
EPOCHS = 100

# Dataset paths
data_dir = "/kaggle/input/road-damage-dataset-potholes-cracks-and-manholes/data"

# Configurazione parametri (run gj4071db)
config = {
    'batch': 16,
    'optimizer': "RAdam",
    'lr0': 0.007613363286285629,
    'lrf': 0.06982332385381142,
    'weight_decay': 0.0071554110355948435,
    'momentum': 0.1586633287328679,
    'dropout': 0.25488093038438164
}

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
def create_yaml():
    """Crea il file YAML che punta direttamente alla tua struttura"""
    yaml_path = "data.yaml"
    
    data = {
        'path': data_dir,
        'train': 'images',
        'val': 'images',  # Usa lo stesso set per train e val
        'nc': 3,
        'names': ['pothole', 'crack', 'manhole']
    }
    
    with open(yaml_path, 'w') as f:
        yaml.dump(data, f)
    
    print(f"✅ File YAML creato: {yaml_path}")
    return yaml_path

In [5]:
def train_model(data_yaml_path, config):
    """Training del modello YOLO"""
    
    print(f"\n🚀 Avvio training con configurazione:")
    for key, value in config.items():
        print(f"   - {key}: {value}")
    
    # Inizializza modello
    model = YOLO("yolo11s.pt")
    
    # Training
    print("\n⏳ Training in corso...")
    results = model.train(
        data=data_yaml_path,
        epochs=EPOCHS,
        batch=config['batch'],
        imgsz=INPUT_SIZE,
        optimizer=config['optimizer'],
        lr0=config['lr0'],
        lrf=config['lrf'],
        weight_decay=config['weight_decay'],
        momentum=config['momentum'],
        dropout=config['dropout'],
        patience=PATIENCE,
        device=0,
        amp=True,
        seed=RANDOM_SEED,
        deterministic=True,
        project=PROJECT,
        name="training",
        exist_ok=True,
        val=True,
        save=True,
        plots=True,
        verbose=True,
        workers=4,
    )
    
    # Validation
    print("\n📊 Validazione finale...")
    val_results = model.val(
        data=data_yaml_path,
        iou=IOU_THRESHOLD,
        conf=CONF_THRESHOLD
    )
    
    metrics = val_results.box
    class_names = ['pothole', 'crack', 'manhole']
    
    # Stampa risultati per classe
    print("\n📈 Risultati per classe:")
    print("-" * 70)
    print(f"{'Classe':<12} {'Precision':<12} {'Recall':<12} {'F1':<12} {'mAP50':<12}")
    print("-" * 70)
    
    for i, name in enumerate(class_names):
        precision = float(metrics.p[i])
        recall = float(metrics.r[i])
        f1 = float(metrics.f1[i])
        ap50 = float(metrics.ap50[i])
        
        print(f"{name:<12} {precision:<12.4f} {recall:<12.4f} {f1:<12.4f} {ap50:<12.4f}")
    
    # Stampa medie
    print("-" * 70)
    print(f"{'MEDIA':<12} {float(metrics.mp):<12.4f} {float(metrics.mr):<12.4f} "
          f"{float(metrics.f1.mean()):<12.4f} {float(metrics.map50):<12.4f}")
    print("-" * 70)
    
    print(f"\n✅ Training completato!")
    print(f"   - mAP50: {float(metrics.map50):.4f}")
    print(f"   - mAP50-95: {float(metrics.map):.4f}")
    print(f"   - Fitness: {float(metrics.fitness()):.4f}")

    #Exporting ONNX
    model.export(format="onnx")
    
    # Cleanup
    del model
    torch.cuda.empty_cache()
    gc.collect()
    
    return val_results

In [6]:
# Crea YAML
data_yaml_path = create_yaml()

# Training
results = train_model(data_yaml_path, config)

print("\n🏁 Processo completato!")

✅ File YAML creato: data.yaml

🚀 Avvio training con configurazione:
   - batch: 16
   - optimizer: RAdam
   - lr0: 0.007613363286285629
   - lrf: 0.06982332385381142
   - weight_decay: 0.0071554110355948435
   - momentum: 0.1586633287328679
   - dropout: 0.25488093038438164

⏳ Training in corso...
Ultralytics 8.3.239 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.25488093038438164, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, 

/usr/local/lib/python3.11/dist-packages/ultralytics/engine/validator.py:298: RuntimeWarning: invalid value encountered in greater_equal
  matches = np.nonzero(iou >= threshold)  # IoU > threshold and classes match


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 63/63 5.6it/s 11.2s
                   all       2009       4737    0.00158      0.021   0.000801   0.000234

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      3/100      4.58G      2.801      3.312      2.124         40        640: 100% ━━━━━━━━━━━━ 126/126 3.0it/s 41.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 63/63 5.0it/s 12.7s
                   all       2009       4737      0.361     0.0476    0.00976    0.00356

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      4/100       4.6G      2.766      3.151      2.104         29        640: 100% ━━━━━━━━━━━━ 126/126 3.1it/s 41.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 63/63 5.1it/s 12.3s
                   all       2009       47

/usr/local/lib/python3.11/dist-packages/ultralytics/engine/validator.py:298: RuntimeWarning: invalid value encountered in greater_equal
  matches = np.nonzero(iou >= threshold)  # IoU > threshold and classes match


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 63/63 5.2it/s 12.2s
                   all       2009       4737      0.218     0.0712     0.0659     0.0231

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      6/100      4.68G      2.637      2.928       2.07         39        640: 100% ━━━━━━━━━━━━ 126/126 3.0it/s 41.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 63/63 5.4it/s 11.6s
                   all       2009       4737      0.208      0.142      0.117     0.0477

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      7/100      4.74G      2.585       2.77      1.994         29        640: 100% ━━━━━━━━━━━━ 126/126 3.1it/s 41.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 63/63 5.0it/s 12.5s
                   all       2009       47

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 63/63 5.2it/s 12.0s
                   all       2009       4737      0.466      0.479      0.461      0.203

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     93/100      5.24G      2.034      1.951      1.676         17        640: 100% ━━━━━━━━━━━━ 126/126 3.1it/s 41.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 63/63 5.2it/s 12.1s
                   all       2009       4737      0.506        0.5      0.491      0.225

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     94/100      5.26G       2.04      1.917      1.677         27        640: 100% ━━━━━━━━━━━━ 126/126 3.1it/s 41.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 63/63 5.2it/s 12.1s
                   all       2009       47

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       2009       4737      0.552      0.536      0.543      0.253
               pothole        795       1261      0.545      0.464      0.479      0.208
                 crack       1375       2519      0.452       0.34      0.352       0.14
               manhole        759        957      0.661      0.803      0.796       0.41
Speed: 0.1ms preprocess, 2.8ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /kaggle/working/pothole-detector-NatureSR/training

📊 Validazione finale...
Ultralytics 8.3.239 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11s summary (fused): 100 layers, 9,413,961 parameters, 0 gradients
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 152.3±21.5 MB/s, size: 85.8 KB)
val: Scanning /kaggle/input/road-damage-dataset-potholes-cracks-and-manholes/data/labels... 2009 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 2009/2009 726.7it/s 2.8s
WARNING ⚠️ val: Cache directory /kaggle/input/r

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       2009       4737      0.654      0.455      0.567      0.298
               pothole        795       1261       0.65      0.358      0.505      0.255
                 crack       1375       2519      0.584      0.249      0.408      0.196
               manhole        759        957      0.728       0.76      0.788      0.444
Speed: 0.5ms preprocess, 3.2ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to /kaggle/working/runs/detect/val

📈 Risultati per classe:
----------------------------------------------------------------------
Classe       Precision    Recall       F1           mAP50       
----------------------------------------------------------------------
pothole      0.6499       0.3577       0.4614       0.5045      
crack        0.5843       0.2489       0.3491       0.4083      
manhole      0.7285       0.7597       0.7437       0.7882      
----------------------------------------------------------------------
MEDIA        0.